# Deterministic did:btc with a Update to a Singleton Beacon

This notebook works through a basic example of creating a deterministic did:btc and then updating it by publishing an update to a singleton beacon. The example uses the regtest network and interfaces with a local bitcoin node over an RPC endpoint.

It is recommended you run a local Bitcoin regtest network using the [Lightning Polar](https://lightningpolar.com/) application which provides a friendly GUI for interfacing with the network.

# 0. Initial Setup

We do some initial setup to create a DID and an unsecuredDocument to be signed

## 0.1. Add libbtc1 python library to path

In [1]:
import sys
import os

notebooks_path = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add the Notebooks directory to the sys.path
sys.path.append(notebooks_path)

## 0.2. Create key pair

In [2]:
from buidl.mnemonic import secure_mnemonic
from buidl.hd import HDPrivateKey


In [3]:
## Run this if you want a new hardware key
mnemonic = secure_mnemonic()

# mnemonic = "prosper can dial lumber write coconut express imitate husband isolate inside release brush media please kind comic pill science repeat basic also endorse bronze"
root_hdpriv = HDPrivateKey.from_mnemonic(mnemonic, network="signet")
print("Mnemonic : ", mnemonic)

Mnemonic :  game torch tell section ketchup ivory lawn guitar burden thought height raven volume diary charge borrow custom purity opera section hope senior chef sea


In [4]:
didkey_purpose = "11"

initial_sk = root_hdpriv.get_private_key(didkey_purpose, address_num=2)
initial_pk = initial_sk.point

print("Secp256k1 PrivateKey", initial_sk.hex())
print("Secp256k1 Public Key", initial_pk.__repr__())

Secp256k1 PrivateKey 011620400ad8008eae2fe54ce94345a6f80b6845705ffb941d719c2e12951f8f
Secp256k1 Public Key S256Point(036eaca63d04f3a8c1b701d63f60e2617e4fd0c4afff23f1d1cc932a1a1cbe3b89)


## 1. Deterministically Create DID BTC1 

In [5]:
from libbtc1.did import create_deterministic

did_btc1, did_document = create_deterministic(initial_pk, network="regtest")

In [6]:
did_btc1

'did:btc1:regtest:k1qdh2ef3aqne63sdhq8tr7c8zv9lyl5xy4llj8uw3ejfj5xsuhcacjq98ccc'

In [7]:
import json
print(json.dumps(did_document, indent=2))

{
  "id": "did:btc1:regtest:k1qdh2ef3aqne63sdhq8tr7c8zv9lyl5xy4llj8uw3ejfj5xsuhcacjq98ccc",
  "@context": [
    "https://www.w3.org/ns/did/v1",
    "https://did-btc1/TBD/context"
  ],
  "verificationMethod": [
    {
      "id": "#initialKey",
      "type": "Multikey",
      "controller": "did:btc1:regtest:k1qdh2ef3aqne63sdhq8tr7c8zv9lyl5xy4llj8uw3ejfj5xsuhcacjq98ccc",
      "publicKeyMultibase": "zQ3shn68faoXE2EqCTtefQXNLgaTa7ZohG2ftZjgXphStJsGc"
    }
  ],
  "authentication": [
    "#initialKey"
  ],
  "assertionMethod": [
    "#initialKey"
  ],
  "capabilityInvocation": [
    "#initialKey"
  ],
  "capabilityDelegation": [
    "#initialKey"
  ],
  "service": [
    {
      "id": "#initialP2PKH",
      "type": "SingletonBeacon",
      "serviceEndpoint": "bitcoin:moFJwqLXBDmw4rnWQm9c3ag4kSdFxD5yiz"
    },
    {
      "id": "#initialP2WPKH",
      "type": "SingletonBeacon",
      "serviceEndpoint": "bitcoin:bcrt1q2n9edlz3yehahctcj6p93lzznhz9m0kzp67ung"
    },
    {
      "id": "#initialP2

## 2 Create DID update payload

- JSON Patch transformation from current document to new one 
- Bump version number

 

In [8]:
# This is saying add linkedDomanSE to the end of the service list of a json document.
import copy
# Adding a new service
linked_domain = {
    "id":"#linked-domain",
    "type": "LinkedDomains", 
    "serviceEndpoint": "https://contact-me.com"    
}


service_path = f"/service/{len(did_document["service"])}"

update_patch = [{'op': 'add', 'path': service_path, 'value': linked_domain}]


print(update_patch)

[{'op': 'add', 'path': '/service/3', 'value': {'id': '#linked-domain', 'type': 'LinkedDomains', 'serviceEndpoint': 'https://contact-me.com'}}]


In [9]:
import jsonpatch

patch = jsonpatch.JsonPatch(update_patch)

v2_did_document = patch.apply(did_document)


In [10]:
v2_did_document

{'id': 'did:btc1:regtest:k1qdh2ef3aqne63sdhq8tr7c8zv9lyl5xy4llj8uw3ejfj5xsuhcacjq98ccc',
 '@context': ['https://www.w3.org/ns/did/v1', 'https://did-btc1/TBD/context'],
 'verificationMethod': [{'id': '#initialKey',
   'type': 'Multikey',
   'controller': 'did:btc1:regtest:k1qdh2ef3aqne63sdhq8tr7c8zv9lyl5xy4llj8uw3ejfj5xsuhcacjq98ccc',
   'publicKeyMultibase': 'zQ3shn68faoXE2EqCTtefQXNLgaTa7ZohG2ftZjgXphStJsGc'}],
 'authentication': ['#initialKey'],
 'assertionMethod': ['#initialKey'],
 'capabilityInvocation': ['#initialKey'],
 'capabilityDelegation': ['#initialKey'],
 'service': [{'id': '#initialP2PKH',
   'type': 'SingletonBeacon',
   'serviceEndpoint': 'bitcoin:moFJwqLXBDmw4rnWQm9c3ag4kSdFxD5yiz'},
  {'id': '#initialP2WPKH',
   'type': 'SingletonBeacon',
   'serviceEndpoint': 'bitcoin:bcrt1q2n9edlz3yehahctcj6p93lzznhz9m0kzp67ung'},
  {'id': '#initialP2TR',
   'type': 'SingletonBeacon',
   'serviceEndpoint': 'bitcoin:bcrt1p6rs5tnq94rt4uu5edc9luahlkyphk30yk8smwfzurpc8ru06vcws8ylq7l'},
 

In [11]:
# This is the first update
# If you want to do another update you MUST bump the version
from buidl.helper import encode_base58, sha256
import jcs

source_hash_bytes = sha256(jcs.canonicalize(did_document))
source_hash = encode_base58(source_hash_bytes)
target_hash_bytes = sha256(jcs.canonicalize(v2_did_document))
target_hash = encode_base58(target_hash_bytes)
targetVersionId = 2


In [12]:




did_update_payload = {
    '@context': [
        'https://w3id.org/security/v2',
        'https://w3id.org/zcap/v1',
        'https://w3id.org/json-ld-patch/v1'
        # TODO did:btc1 zcap context
    ],
    'patch': update_patch,
    # TODO: this might not go here?
    'sourceHash': source_hash,
    'targetHash': target_hash,
    'targetVersionId': targetVersionId
};

## C3. Create DID Update Invocation

### C3.1 Deterministically Generate Root Capability for DID document

This is a root capability for a specific did:btc1 to invoke a capability to update that did:btc's DID document

In [13]:
import urllib
root_capability = {
  "@context": "https://w3id.org/security/v2",
  "id": f"urn:zcap:root:{urllib.parse.quote(did_btc1)}",
  "controller": did_btc1,
  "invocationTarget": did_btc1
};
root_capability

{'@context': 'https://w3id.org/security/v2',
 'id': 'urn:zcap:root:did%3Abtc1%3Aregtest%3Ak1qdh2ef3aqne63sdhq8tr7c8zv9lyl5xy4llj8uw3ejfj5xsuhcacjq98ccc',
 'controller': 'did:btc1:regtest:k1qdh2ef3aqne63sdhq8tr7c8zv9lyl5xy4llj8uw3ejfj5xsuhcacjq98ccc',
 'invocationTarget': 'did:btc1:regtest:k1qdh2ef3aqne63sdhq8tr7c8zv9lyl5xy4llj8uw3ejfj5xsuhcacjq98ccc'}

### C3.2 Invoke root capability over the DID update payload

**Note:** There are no libraries for this in python. I have a separate POC that demonstrates how to achieve this using libraries for Digital Bazaar in JavaScript.

In [19]:
from di_bip340.multikey import SchnorrSecp256k1Multikey
from di_bip340.data_integrity_proof import DataIntegrityProof
from di_bip340.cryptosuite import Bip340JcsCryptoSuite

did_update_invocation = copy.deepcopy(did_update_payload)

multikey = SchnorrSecp256k1Multikey(id="#initialKey", controller=did_btc1, private_key=initial_sk)
cryptosuite = Bip340JcsCryptoSuite(multikey)
di_proof = DataIntegrityProof(cryptosuite)



options = {
        "type": "DataIntegrityProof",
        "cryptosuite": "bip340-jcs-2025",
        "verificationMethod": multikey.full_id(),
        "proofPurpose": "capabilityInvocation",
        "capability": root_capability["id"],
        "capabilityAction": "Write"
}


secured_did_update_payload = di_proof.add_proof(did_update_invocation, options)

In [22]:
print(secured_did_update_payload)

{'@context': ['https://w3id.org/security/v2', 'https://w3id.org/zcap/v1', 'https://w3id.org/json-ld-patch/v1'], 'patch': [{'op': 'add', 'path': '/service/3', 'value': {'id': '#linked-domain', 'type': 'LinkedDomains', 'serviceEndpoint': 'https://contact-me.com'}}], 'sourceHash': '9kSA9j3z2X3a26yAdJi6nwg31qyfaHMCU1u81ZrkHirM', 'targetHash': 'C45TsdfkLZh5zL6pFfRmK93X4EdHusbCDwvt8d7Xs3dP', 'targetVersionId': 2, 'proof': {'type': 'DataIntegrityProof', 'cryptosuite': 'bip340-jcs-2025', 'verificationMethod': 'did:btc1:regtest:k1qdh2ef3aqne63sdhq8tr7c8zv9lyl5xy4llj8uw3ejfj5xsuhcacjq98ccc#initialKey', 'proofPurpose': 'capabilityInvocation', 'capability': 'urn:zcap:root:did%3Abtc1%3Aregtest%3Ak1qdh2ef3aqne63sdhq8tr7c8zv9lyl5xy4llj8uw3ejfj5xsuhcacjq98ccc', 'capabilityAction': 'Write', '@context': ['https://w3id.org/security/v2', 'https://w3id.org/zcap/v1', 'https://w3id.org/json-ld-patch/v1'], 'proofValue': 'z3yfzVGdoDF4s8y4Bk8JeV9XuZw1nMeMtNW3x5brEm7DNtmWZkNBPbCLzUBJRpctBj9QJL1dydm94ZNsPxosPnkPP

# Hash invocation

Note: this might end up being an ipfs CID

In [23]:
canonical_bytes = jcs.canonicalize(secured_did_update_payload)
invocation_hash = sha256(canonical_bytes)

## Get beacon address

In [24]:
beacon_address = initial_pk.p2wpkh_address(network="regtest")
beacon_address

'bcrt1q2n9edlz3yehahctcj6p93lzznhz9m0kzp67ung'

## Connect to local RegTest Node

In [25]:
import asyncio

from bitcoinrpc import BitcoinRPC

In [26]:
rpc = BitcoinRPC.from_config("http://localhost:18443", ("polaruser", "polarpass"))
await rpc.getconnectioncount()

0

In [27]:
bitcoin_balance = await rpc.acall("getbalance", {})

In [28]:
bitcoin_balance

3347.1898026

## Fund Beacon address

This makes an RPC call to the Bitcoin node and telling it to use its controlled funds to spend a Bitcoin transaction to a specific address with the value of 0.2 BTC. The result returns the bitcoin transaction ID.

In [29]:
res = await rpc.acall("send", {"outputs": { beacon_address: 0.2}})

In [30]:
funding_txid = res["txid"]
funding_txid

'a4646b7d0d429a7f4c73d9e086a5b74c147e64219751a334c2767abdea7c5a89'

In [31]:
funding_tx_hex = await rpc.acall("getrawtransaction", {"txid": funding_txid})
funding_tx_hex

'02000000000101324824647d208d021a5c04c92634801e4ad451b58c5922c8ef0223b8bb044b950000000000fdffffff02a4ec451901000000160014e1788366ee8e07892b833e6cfef7a80ba10d35f9002d31010000000016001454cb96fc51266fdbe178968258fc429dc45dbec20247304402206a92ddcb98a29f0b5dfee273e0e78b70390491f7989100599badc077c49deb47022054ee8f4a39b6b476fc941c34defc9b14dfc405ee3667e816b4a536014ac35d94012103a601ea5609c246a1c477b4cfd1fc23ffaeaf742d8bd45d55c943f2581d83102b00000000'

In [32]:
from buidl.tx import Tx
funding_tx = Tx.parse_hex(funding_tx_hex)
funding_tx


tx: a4646b7d0d429a7f4c73d9e086a5b74c147e64219751a334c2767abdea7c5a89
version: 2
locktime: 0
tx_ins:
954b04bbb82302efc822598cb551d44a1e803426c9045c1a028d207d64244832:0
tx_outs:
4718980260:OP_0 e1788366ee8e07892b833e6cfef7a80ba10d35f9 
20000000:OP_0 54cb96fc51266fdbe178968258fc429dc45dbec2 

## Construct Beacon Signal

In [33]:
from buidl.tx import Tx, TxIn, TxOut, SIGHASH_DEFAULT

# TODO: Need to fund a beacon address
prev_tx = bytes.fromhex(funding_txid)  # Identifying funding tx
prev_index = 1 # Identify funding output index


tx_in = TxIn(prev_tx=prev_tx, prev_index=prev_index)

# Hack the TxIn to know about the TxOut it is spending
# This is to do with checks in the Buidl library 
tx_in._script_pubkey = funding_tx.tx_outs[prev_index].script_pubkey
tx_in._value = funding_tx.tx_outs[prev_index].amount

# from buidl.tx import URL
# URL["regtest"] = "http://localhost:18443"

print("Tx Input satoshis: ",tx_in.value(network="regtest"))

Tx Input satoshis:  20000000


In [34]:
from buidl.script import ScriptPubKey

# This is the TxOutput of the SignletonBeacon beacon signal [OP_RETURN, OP_PUSH_32, <invocation_hash>]
# It announces and attests to an update through the invocation_hash
script_pubkey = ScriptPubKey([0x6a, invocation_hash])

beacon_signal_txout = TxOut(0, script_pubkey)

In [35]:
tx_fee = 350

refund_amount = tx_in.value() - tx_fee

script_pubkey = initial_pk.p2wpkh_script()
refund_out = TxOut(amount=refund_amount, script_pubkey=script_pubkey)

In [36]:
tx_ins = [tx_in]

tx_outs = [beacon_signal_txout,refund_out]

pending_beacon_signal = Tx(version=1, tx_ins=tx_ins, tx_outs=tx_outs, network="regtest",segwit=True)


print(pending_beacon_signal.serialize().hex())

01000000000101895a7ceabd7a76c234a3519721647e144cb7a586e0d9734c7f9a420d7d6b64a40100000000ffffffff020000000000000000226a20335a39b28577022748ff4ce580a3a9e17d8824d6bb73fd01b6701e1564a8e01aa22b31010000000016001454cb96fc51266fdbe178968258fc429dc45dbec20000000000


## Sign Beacon Signal

In [37]:
pending_beacon_signal.sign_input(0, initial_sk)

True

In [38]:
signed_hex = pending_beacon_signal.serialize().hex()

## Broadcast Beacon Signal

In [39]:
signal_id = await rpc.acall("sendrawtransaction", {"hexstring": signed_hex})

In [40]:
signal_id

'5f8cd13f39fa509b1cdfdc7c6588b6cda99e82202e9498dff9f37dc99d4a1e10'

In [41]:
sidecar_data = {
    "did": did_btc1,
    "signalsMetadata": {}

}

In [42]:
sidecar_data["signalsMetadata"][signal_id] = {
    "updatePayload": secured_did_update_payload
}

In [43]:
print(json.dumps(sidecar_data, indent=2))

{
  "did": "did:btc1:regtest:k1qdh2ef3aqne63sdhq8tr7c8zv9lyl5xy4llj8uw3ejfj5xsuhcacjq98ccc",
  "signalsMetadata": {
    "5f8cd13f39fa509b1cdfdc7c6588b6cda99e82202e9498dff9f37dc99d4a1e10": {
      "updatePayload": {
        "@context": [
          "https://w3id.org/security/v2",
          "https://w3id.org/zcap/v1",
          "https://w3id.org/json-ld-patch/v1"
        ],
        "patch": [
          {
            "op": "add",
            "path": "/service/3",
            "value": {
              "id": "#linked-domain",
              "type": "LinkedDomains",
              "serviceEndpoint": "https://contact-me.com"
            }
          }
        ],
        "sourceHash": "9kSA9j3z2X3a26yAdJi6nwg31qyfaHMCU1u81ZrkHirM",
        "targetHash": "C45TsdfkLZh5zL6pFfRmK93X4EdHusbCDwvt8d7Xs3dP",
        "targetVersionId": 2,
        "proof": {
          "type": "DataIntegrityProof",
          "cryptosuite": "bip340-jcs-2025",
          "verificationMethod": "did:btc1:regtest:k1qdh2ef3aqne63sdh